In [1]:
import nltk
#import sqlite3
import re
#import json
from nltk import *
from nltk.tag import *
from nltk.corpus import stopwords
from nltk.sentiment import *
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment.util import *
import pandas as pd

In [2]:
#nltk.download('vader_lexicon')

In [2]:
n_instances = 5000
subj_docs = [(sent, 'subj') for sent\
             in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent\
            in subjectivity.sents(categories='obj')[:n_instances]]

In [3]:
train = int(n_instances * 0.8)
test = int(n_instances * 0.2)

In [4]:
train

4000

In [5]:
train_subj_docs = subj_docs[:train]
test_subj_docs = subj_docs[test:]
train_obj_docs = obj_docs[:train]
test_obj_docs = obj_docs[test:]

In [6]:
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc)\
                                           for doc in training_docs])

In [7]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg,
                                                   min_freq=4)

In [8]:
#unigram_feats

In [9]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
sid = SentimentIntensityAnalyzer()

Training classifier


In [18]:
TRACETOPIC = ['trade',
 'labor',
 'nafta',
 'agreement',
 'union',
 'manufactur',
 'worker',
 'china',
 'wage',
 'plant',
 'corpor',
 'canada',
 'play',
 'bargain',
 'unfair']

In [14]:
ls = ['trade', 'labor', 'worker', 'agreement', 'union', 'nafta', 'china', 'manufactur', 'play', 'plant', 'outsourc', 'wage',
'steel', 'organ', 'green','trade', 'agreement', 'nafta', 'worker', 'manufactur', 
 'china', 'labor', 'deal', 'union', 'plant', 'green', 'steel', 'interest', 'special', 'oversea' ]

In [15]:
topic = set(ls)

In [17]:
#topic

In [32]:
df = pd.read_csv('../../Data/alldf_w15_180306.csv')

In [36]:
df1 = pd.read_csv('../../Data/alldf_t100_frexscore_swing.csv')

In [40]:
df2 = df1.merge(df, on = ['Author','Year','No.','Date'])

In [42]:
#df2

In [60]:
rsw = df2[(df2['Rustbelt_x'] == 1)&(df2['swing'] ==1)].reset_index()
rnsw = df2[(df2['Rustbelt_x'] == 1)&(df2['swing'] ==0)].reset_index()
nrsw = df2[(df2['Rustbelt_x'] == 0)&(df2['swing'] ==1)].reset_index()
nrnsw = df2[(df2['Rustbelt_x'] == 0)&(df2['swing'] ==0)].reset_index()

In [61]:
#rsw.shape

In [62]:
#df.loc[1064]['Clean']

In [130]:
commentls = df.loc[1064]['Clean'].split('. ')
commentls1 = df.loc[62]['Clean'].split('. ')

In [132]:
#commentls1

In [102]:
def run_sentiment(comment):
    commentls = comment.split('. ')
    neg_list = []
    pos_list = []
    num_sentence = 0
    score_d = {}
    negtradels = []
    negnotradels = []
    postradels = []
    posnotradels = []
    for i in range(len(commentls)):
        num_sentence += 1
        sentence = commentls[i]
        ss = sid.polarity_scores(sentence)
        score_d[num_sentence] = {'score':ss, 'sentence': sentence}

        if ss['neg'] > ss['neu'] or ss['neg'] > ss['pos']:
            neg_list.append(sentence)
        else:
            pos_list.append(sentence)
            
        negtradels = set()
        negnotradels = set()  
        for sent in neg_list:
            for word in topic:
                if word in sent:
                    negtradels.add(sent)
                    continue
                else:
                    negnotradels.add(sent)
                    continue
                    
        postradels = set()
        posnotradels = set()                   
        for sent in pos_list:

            for word in topic:
                if word in sent:
                    postradels.add(sent)
                    continue
                else:
                    posnotradels.add(sent)
                    continue
    return (score_d, neg_list, pos_list, negtradels, negnotradels, postradels, posnotradels)    

In [103]:
#rust = pd.read_csv('../../Data/rust_frex_text.csv')

In [104]:
#rust

In [105]:
#norust = pd.read_csv('../../Data/norust_frex_text.csv')

In [106]:
#rsw['Clean_x']

In [107]:
dfls = []
for i in range(len(rsw)):
    result = run_sentiment(rsw['Clean_x'].loc[i])
    dfls.append([list(result[3]),list(result[4]),list(result[5]),list(result[6])])    

In [108]:
rustsentdf = pd.DataFrame(dfls)

In [109]:
rustsentdf.rename(columns = {0:'neg_trade', 1:'neg_notrade', 2:'pos_trade', 3:'pos_notrade'},inplace = True)
rust_sentiment = rsw.join(rustsentdf)

In [111]:
rust_sentiment2 = rust_sentiment.sort_values('pro_rust', ascending=False)

In [172]:
dfls2 = []
for i in range(len(rnsw)):
    result2 = run_sentiment(rnsw['Clean_x'].loc[i])
    dfls2.append([list(result2[3]),list(result2[4]),
                  list(result2[5]),list(result2[6])])    

rustsentdfnsw = pd.DataFrame(dfls2)
rustsentdfnsw.rename(columns = {0:'neg_trade', 1:'neg_notrade', 2:'pos_trade', 3:'pos_notrade'},inplace = True)
rust_sentiment = rnsw.join(rustsentdfnsw)

In [163]:
rustsentdfnsw2= rust_sentiment.sort_values('pro_rust', ascending=False)

In [114]:
dfls3 = []
for i in range(len(nrsw)):
    result2 = run_sentiment(nrsw['Clean_x'].loc[i])
    dfls3.append([list(result2[3]),list(result2[4]),
                  list(result2[5]),list(result2[6])])    

norustsentdfsw = pd.DataFrame(dfls3)
norustsentdfsw.rename(columns = {0:'neg_trade', 1:'neg_notrade', 2:'pos_trade', 3:'pos_notrade'},inplace = True)
norust_sentimentsw = nrsw.join(norustsentdfsw)

In [122]:
norust_sentimentsw2 = norust_sentimentsw.sort_values('pro_norust', ascending=False)

In [115]:
dfls4 = []
for i in range(len(rnsw)):
    result2 = run_sentiment(rnsw['Clean_x'].loc[i])
    dfls4.append([list(result2[3]),list(result2[4]),
                  list(result2[5]),list(result2[6])])    

norustsentdfns = pd.DataFrame(dfls4)
norustsentdfns.rename(columns = {0:'neg_trade', 1:'neg_notrade', 2:'pos_trade', 3:'pos_notrade'},inplace = True)
norust_sentimentns = rnsw.join(norustsentdfns)

In [174]:
#rustsentdfnsw2

In [180]:
def title(df,i):
    title = str(df['Year'].iloc[i]) +'  '\
+ str(df['Author'].iloc[i])+' '\
+ df['Title_x'].iloc[i]

    return title

In [228]:
title( rust_sentiment2, 3)

'2008  Barack Obama Remarks to the AFL-CIO in Philadelphia'

In [226]:
for sentence in rust_sentiment2['neg_trade'].iloc[3]:
    if 'trade' in sentence:
        print(sentence)

And it's an administration that denies labor a seat at the table when trade deals are being negotiated, that doesn't believe in unions, that doesn't believe in organizing, and that's packed the labor relations board with their corporate buddies


In [227]:
for sentence in rust_sentiment2['pos_trade'].iloc[3]:
    if 'trade' in sentence:
        print(sentence)

But what I refuse to accept is that we have to sign trade deals like the South Korea Agreement that are bad for American workers
One hundred and eighty-one years ago, in the fall of 1827, a group of mechanics met in the shadow of Independence Hall to form what they called the Mechanics Union of Trade Associations - a moment that marked the birth of the trade union movement in America
Like George Bush, Senator McCain is committed to more tax cuts for the rich, and more trade agreements that fail to protect American workers
What I oppose - and what I have always opposed - are trade deals that put the interests of multinational corporations ahead of the interests of Americans workers - like NAFTA, and CAFTA, and permanent normal trade relations with China
So you can trust me when I say that whatever trade deals we negotiate when I'm President will be good for American workers, and that they'll have strong labor and environmental protections that we'll enforce


In [121]:
norust_sentimentns2 = norust_sentimentns.sort_values('pro_norust', ascending=False)

In [60]:
noresult = run_sentiment(norust['Clean'].loc[0])

In [62]:
noresult1 = run_sentiment(norust['Clean'].loc[1])

In [66]:
#noresult1[0]

In [55]:
rust_sentiment